intalace knihoven

In [ ]:
%pip install aiohttp
%pip install asyncio
%pip install openpyxl
%pip install plotly
%pip install nbformat
%pip install matplotlib seaborn


Importování knihoven

In [ ]:
import aiohttp
import asyncio
import json
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

Získání přihlašovacích údajů

In [ ]:
async def getToken(username, password):
    keyurl = "http://127.0.0.1:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            # print(resp.status)
            keyJson = await resp.json()
            # print(keyJson)

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            # print(resp.status)
            tokenJson = await resp.json()
            # print(tokenJson)
    return tokenJson.get("token", None)
            


Funkce pro query

In [ ]:
def query(q, token):
    async def post(variables):
        gqlurl = "http://127.0.0.1:33001/api/gql"
        payload = {"query": q, "variables": variables}
        # headers = {"Authorization": f"Bearer {token}"}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            # print(headers, cookies)
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                # print(resp.status)
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post 


Nějaký kód od hrbloka

In [ ]:
from itertools import product
from functools import reduce

def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            # print("flatList", row)
            yield row
            
def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    # print("flatDict.result", result)
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                     
    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced
        
def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"


Definice
    - přihlašovacích údajů
    - query pro získání dat
    - mapování dat

In [ ]:
username = "john.newbie@world.com"
password = "john.newbie@world.com"

queryStr = """
{
  result: eventPage(
    where: {
      _and: [
        {type_id: {_in: [
          "b87d7be6-8fd4-11ed-a6d4-0242ac110002",
          "b87d3ff0-8fd4-11ed-a6d4-0242ac110002"]}},
        {startdate: {_gt: "2022-09-01T00:00:00"}},
        {enddate: {_lt: "2023-09-01T00:00:00"}}
      ]
    }
  ) {
    id
    name
    startdate
    enddate
    eventType {
      id
      name
    }
    groups {
      id
      name
    }
  }
}
"""

mappers = {

    "groups_name": "groups.name",
    "eventType_name": "eventType.name",
    "startdate": "startdate",
    "enddate": "enddate"
    # "id": "id",
    # "name": "name",
    # "startdate": "startdate",
    # "enddate": "enddate",
    # "eventTypeId": "eventType.id",
    # "eventTypeName": "eventType.name",
    # # "eventduration": "presences.event.duration",
    # "groupsId": "groups.id",
    # "groups_name": "groups.name"
}

async def fullPipe():
    # global pandasData
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data", None)
    result = data.get("result", None)
  # response = await qfunc({})
    flatData = flatten(result, {}, mappers)
    pandasData = pd.DataFrame(flatData)

  # transforamce a agregace dat
  # agregace enddate a startdate na duration
    pandasData['startdate'] = pd.to_datetime(pandasData['startdate'])
    pandasData['enddate'] = pd.to_datetime(pandasData['enddate'])

    pandasData['duration'] = (pandasData['enddate'] - pandasData['startdate']).dt.total_seconds() / 60

    aggregatedData = pandasData.groupby(['groups_name', 'eventType_name',])['duration'].sum().reset_index()

  # uložení výsledků z dat na UOIS
    with open ('aggregatedData.json', 'w', encoding='utf-8') as outputfile:
        json.dump(aggregatedData.to_dict(orient='records'), outputfile)

    # return aggregatedData

  # otevření a načtení vlastních dat

    with open('aggregated.json', 'r') as file:
      real_data = json.load(file)

  # převod na tabulku pro excel
    df = pd.DataFrame(real_data)
    df.to_excel('aggregatedData.xlsx', index=False)

    # print(df)

  # vytvoření kont. tabulky (stejně je to normální tabulka bo nevím)
    pivot_table = pd.pivot_table(
    df,
    values='total_duration',
    index=['group_name'],
    columns=['eventtype_name'],
    aggfunc='sum',
    fill_value=0
)

  # Print the pivot table
    # print(pivot_table)

  # Save the pivot table to an Excel file
    pivot_table.to_excel('aggregated_pivot_table.xlsx', engine='openpyxl')

  # vytvoření sloupcového grafu

    plt.figure(figsize=(12, 8))
    sns.barplot(data=df, x='group_name', y='total_duration', hue='eventtype_name')
    plt.title('Total Duration of Events by Group and Event Type')
    plt.xlabel('Group Name')
    plt.ylabel('Total Duration (minutes)')
    plt.xticks(rotation=45)
    plt.legend(title='Event Type')
    plt.tight_layout()

    bar_plot = plt.gcf()

  # vytvoření sunburst graph

    fig = px.sunburst(
    df,
    path=['eventtype_name', 'group_name'],
    values='total_duration',
    title='Total duration of events for groups'
    )

    # fig.show()

    return df, pivot_table, bar_plot, fig

In [ ]:
df, pivot_table, bar_plot, fig = await fullPipe()
print(df)
print("\n\n\n")
print(pivot_table)
bar_plot.show()
fig.show()